# 다항회귀와 과대적합/과소적합의 이해

- 독립변수(feature)와 종속변수(target)가 하나일순 없다.
- 회귀가 독립변수의 단항식이 아닌 2,3차 방정식과 같은 다항식으로 표현된 것을 다항(polynomial)회귀라고 한다.
- 다항회귀는 비선형 회귀가 아니라 선형회귀임을 주의
- 사이킷런은 다항 회귀를 위한 클래스를 제공하지 않아서 비선형 함수를 선형 모델에 적용시키는 방법으로 구현한다.
    - 사이킷런의 PolynomialFeatures클래스를 통해 feature를 Polynomial(다항식) feature로 변환한다.

In [1]:
from sklearn.preprocessing import PolynomialFeatures
import numpy as np

In [2]:
# 다항식으로 변환한 단항식 생성
X = np.arange(4).reshape(2,2)
print(X)

[[0 1]
 [2 3]]


In [4]:
# PolynomialFeatures로 degree=2 인 2차 다항식으로 변환
# degree는 몇차로 바꿀건지 설정
poly = PolynomialFeatures(degree=2)
poly.fit(X)
poly_ftr = poly.transform(X)
print('변한된 2차 다항식 계수 피처:\n', poly_ftr)

변한된 2차 다항식 계수 피처:
 [[1. 0. 1. 0. 0. 1.]
 [1. 2. 3. 4. 6. 9.]]


## 3차 다항회귀 결정 함수를 임의로 설정하고 회귀 계수로 예측하기

In [7]:
def polynomial_func(X):
    y = 1 + 2*X[:,0] + 3*X[:,0]**2 + 4*X[:,1]**3
    return y

X = np.arange(4).reshape(2,2)
print('일차 단항식 계수 feature:\n', X)

y = polynomial_func(X)
print('삼차 다항식 결정값:\n', y)

일차 단항식 계수 feature:
 [[0 1]
 [2 3]]
삼차 다항식 결정값:
 [  5 125]
